In [23]:
import sys
import os
import os.path as path

import random
from scipy.stats import beta
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.cm as cm
import matplotlib.pyplot as plt

sys.path.insert(1, path.join(path.abspath(path.pardir)))
import sims
from platform_opt import *

In [24]:
# remove determinism
seed = 42
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)

## Load datasets

In [25]:
# load probabilities
df = pd.read_csv("csv/abortion_new_graph.csv")

In [31]:
df.groupby('User').count().Follower.median()

27.0

In [4]:
users = df[['User', 'Group1']].drop_duplicates()
users.groupby('Group1').count()

,User
Group1,
A,196
B,228


## Parameters in the real network

In [5]:
users.groupby('Group1').count() / users.User.count()

,User
Group1,
A,0.462264
B,0.537736


## Propagation of the articles

In [6]:
# GLOBAL VARIABLES
TRIALS = 100
T      = 10
M      = 69


# indices
Gr = {'A': -1, 'B': 1}
Pr = {'Pa': -1, 'Pb': 1}
OPr = {'Pa': 1, 'Pb': -1}
OGr = {'A': 'B', 'B': 'A'}
OAr = {'Pa': 'Pb', 'Pb': 'Pa'}

In [7]:
# UTIL METHODS

## plotting
def plot(df):
    '''Plot the lines in the dataframe.'''
    return sns.lineplot(x='t', y='n', data=df)


def plotgen(g):
    '''Plot the dataframe built from the generator.'''
    df = pd.DataFrame(g, columns=['t', 'nA', 'nB'])
    return plot(df)


## other
def filter_list(l, blacklist=[]):
    '''Filter out from a list the elements in `blacklist`.'''
    return [e for e in l if e not in blacklist]

### Propagation with the model

In [8]:
SAVED_COLS = ['Setting', 'Exposure', 'Theta_A', 'Seed', 'Article', 't', 'N', 'B', 'A',]

In [9]:
# model parameters
epsilon=0.1
new_pi = {-1: 0.5, 1: 0.5}
theta_opt = {1: 0, -1: 1}
theta_half = {1: 0.5, -1: 0.5}

pi, beta_dist, P, v, c, q = sims.get_params('twitter_uselections')
theta_fair = opt_constrained(pi, q, T, epsilon, c, v, beta_dist, exposure_e=0.1)

In [10]:
group_cols = ['Aa', 'Ab', 'Ba', 'Bb']
exps = ['fair', 'opt', 'half']

def run_model_sim(theta, T, pi, M, P, beta_dist, v, c, q, seed=0):
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    
    num_players = np.zeros((T, 4))
    for j, a in enumerate(sims.runModel(theta, T, pi, M, P, beta_dist, v, c, q,)[1]):
        num_players[:, j] =  np.array(a + [0] * (T - len(a))).reshape(-1)
    df = pd.DataFrame(num_players, columns=group_cols)
    df.loc[:, 't'] = range(T)
    return df.astype(int)

In [11]:
# run model simulations
model_results = pd.DataFrame(columns=['Exposure', 'Theta_A', 'Size', 'Seed', 't'] + group_cols)
for exp, theta in zip(exps, [theta_fair, theta_opt, theta_half]):
    params = theta, T, new_pi, 2 * M, P, beta_dist, v, c, q
    for trial in range(TRIALS):
        num_players = run_model_sim(*params, seed=trial)
        num_players.loc[:, ['Exposure', 'Theta_A', 'Size', 'Seed']] = exp, theta[-1], M, trial
        model_results = pd.concat((model_results, num_players))

In [12]:
ids = [c for c in model_results.columns if c not in group_cols]
dfA = pd.melt(model_results, id_vars=ids, value_vars=['Aa', 'Ab']).rename(columns={'value': 'A'})
dfB = pd.melt(model_results, id_vars=ids, value_vars=['Ba', 'Bb']).rename(columns={'value': 'B'})
dfB['A'] = dfA.A
dfB.loc[:, 'Article'] = dfB.variable.str[1]

model_results = dfB.drop('variable', axis=1)
model_results['N'] = model_results.A + model_results.B
model_results['Setting'] = 'model'
model_results[SAVED_COLS].to_csv('results/abortion_results_model.csv', index=False)

### Propagation with the labeled network

In [13]:
def total_num_players(G):
    return len(G.User.unique())


def nplayers_unique(G, group):
    return (G[['User', 'Group1']].drop_duplicates()
                                .query(f'Group1 == "{group}"')
                                .shape[0])


def group_num_players(G):
    nA = nplayers_unique(G, 'A')
    nB = nplayers_unique(G ,'B')
    return nA, nB


def sample_seeds(df, group, size):
    '''Return dataframe with a random sample of users.'''
    # sample `size` users as the first set of users to see the article
    # we need to drop duplicates as we have a row for each (user, follower) pair
    seeds = (df[['User', 'Group1']].drop_duplicates()
                                   .query(f'Group1 == "{group}"')
                                   .sample(size)
                                   .User)
    # take the rows of the seed users
    G = df[df.User.isin(seeds)]
    return G


def did_share(G, article, group, sample_beta=True):
    group_members = G[G.Group1 == group]
    if sample_beta:
        beta_index = Pr[article], Gr[group]
        probs = np.random.beta(*beta_dist[beta_index], size=len(group_members))
    else:
        probs = group_members[article]
    shared = np.random.binomial(1, p=probs).astype(bool)
    return shared


def propagate_single(G, group, article,
                     take_one=True, sample_beta=True, norepetitions=True, steps=T, func=group_num_players):
    '''Evaluates a function on each step of the propagation of an article through the network.'''
    blacklist = set([])
    for t in range(steps):
        yield func(G)
        if not G.empty:
            blacklist.update(set(G.User.tolist()))
            newG = G.copy()
            if take_one: # take one friend for each user
                newG = newG.groupby('User').sample(1)
                # What happens if two different users are followed by the same person?
                # It's more likely that the artcile will be shared.
            # Note: we assume that if someone follows n users sharing the article, there are n toss coins for sharing it.
            
             # toss a coin for each exposed user to determine whether or not they will share the article
            is_member = newG.Group1 == group
            newG.loc[is_member, 'Shared'] = did_share(newG, article, group, sample_beta)
            newG.loc[~is_member, 'Shared'] = did_share(newG, article, OGr[group], sample_beta)
            
            # take one friend for each user
            newG = newG[newG.Shared].Follower.unique()
            if norepetitions:
                newG = filter_list(newG, blacklist=blacklist)
            G = df[df.User.isin(newG)].copy()


def simulate_exposure(theta, df, size, seed, group='A', article='Pa', **kwargs):
    '''Simulates article propagation given theta exposure parameter.'''
    # make deterministic:
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    
    # exposed a fraction of the users to the opposite article
    a_exposed = int(theta * size)
    Ga = sample_seeds(df, group, a_exposed).copy()
    Ga_users = df.User.isin(Ga.User.unique())
    Gb = sample_seeds(df[~Ga_users], group, size - a_exposed).copy()
    #print(group, article, theta, a_exposed, size, total_num_players(Ga), total_num_players(Gb))
    
    # we propagate article a through G1 and article b through G2
    genGa = propagate_single(Ga, group, 'Pa', **kwargs)
    genGb = propagate_single(Gb, group, 'Pb', **kwargs)
    for t, ((na1, nb1), (na2, nb2)) in enumerate(zip(genGa, genGb)):
        # aggregate the number of players in A and B
        yield group, article[-1:], theta, size, seed, t, na1 + na2, nb1 + nb2

In [14]:
def repeat_propagation(theta, df, size, trials, **kwargs):
    '''Method to repeat the experiment a number of times given a seed.'''
    for trial in range(trials):
        for result in simulate_exposure(theta, df, size, seed=trial, **kwargs):
            yield result

In [15]:
# run experiments
COLS = ['Group', 'Article', 'Theta_A', 'Size', 'Seed', 't', 'A', 'B']

# fair (constrained)
dfa0 = pd.DataFrame(repeat_propagation(theta_fair[-1], df, M, TRIALS, group='A', article='Pa'), columns=COLS)
dfb0 = pd.DataFrame(repeat_propagation(theta_fair[1], df, M, TRIALS, group='B', article='Pb'), columns=COLS)
df0 = pd.concat((dfa0, dfb0))
df0['Exposure'] = 'fair'

# half exposure
dfahalf = pd.DataFrame(repeat_propagation(theta_half[-1], df, M, TRIALS, group='A', article='Pa'), columns=COLS)
dfbhalf = pd.DataFrame(repeat_propagation(theta_half[1], df, M, TRIALS, group='B', article='Pb'), columns=COLS)
dfhalf = pd.concat((dfahalf, dfbhalf))
dfhalf['Exposure'] = 'half'

# optimal (unconstrained)
dfaopt = pd.DataFrame(repeat_propagation(theta_opt[-1], df, M, TRIALS, group='A', article='Pa'), columns=COLS)
dfbopt = pd.DataFrame(repeat_propagation(theta_opt[1], df, M, TRIALS, group='B', article='Pb'), columns=COLS)
dfopt = pd.concat((dfaopt, dfbopt))
dfopt['Exposure'] = 'opt'

network_results = pd.concat((df0, dfhalf, dfopt))
network_results['N'] = network_results.A + network_results.B
network_results['Setting'] = 'network'
network_results[SAVED_COLS].to_csv('results/abortion_results_network.csv', index=False)

In [16]:
(pd.concat((network_results[SAVED_COLS],
            model_results[SAVED_COLS]))
     .to_csv('results/abortion_results.csv', index=False))

### Drop Assumption 1: take only one follower

In [17]:
# run experiments
COLS = ['Group', 'Article', 'Theta_A', 'Size', 'Seed', 't', 'A', 'B']

# fair (constrained)
dfa0 = pd.DataFrame(repeat_propagation(theta_fair[-1], df, M, TRIALS, group='A', take_one=False, article='Pa'), columns=COLS)
dfb0 = pd.DataFrame(repeat_propagation(theta_fair[1], df, M, TRIALS, group='B', take_one=False, article='Pb'), columns=COLS)
df0 = pd.concat((dfa0, dfb0))
df0['Exposure'] = 'fair'

# half exposure
dfahalf = pd.DataFrame(repeat_propagation(theta_half[-1], df, M, TRIALS, group='A', take_one=False, article='Pa'), columns=COLS)
dfbhalf = pd.DataFrame(repeat_propagation(theta_half[1], df, M, TRIALS, group='B', take_one=False, article='Pb'), columns=COLS)
dfhalf = pd.concat((dfahalf, dfbhalf))
dfhalf['Exposure'] = 'half'

# optimal (unconstrained)
dfaopt = pd.DataFrame(repeat_propagation(theta_opt[-1], df, M, TRIALS, group='A', take_one=False, article='Pa'), columns=COLS)
dfbopt = pd.DataFrame(repeat_propagation(theta_opt[1], df, M, TRIALS, group='B', take_one=False, article='Pb'), columns=COLS)
dfopt = pd.concat((dfaopt, dfbopt))
dfopt['Exposure'] = 'opt'

network_results = pd.concat((df0, dfhalf, dfopt))
network_results['N'] = network_results.A + network_results.B
network_results['Setting'] = 'network'
network_results[SAVED_COLS].to_csv('results/abortion_results_network_not_takeone.csv', index=False)

In [18]:
(pd.concat((network_results[SAVED_COLS],
            model_results[SAVED_COLS]))
     .to_csv('results/abortion_results_not_takeone.csv', index=False))

### Drop Assumption 2: the same user cannot appear more than once

In [19]:
# run experiments
COLS = ['Group', 'Article', 'Theta_A', 'Size', 'Seed', 't', 'A', 'B']

# fair (constrained)
dfa0 = pd.DataFrame(repeat_propagation(theta_fair[-1],df, M, TRIALS, group='A', norepetitions=False, article='Pa'), columns=COLS)
dfb0 = pd.DataFrame(repeat_propagation(theta_fair[1],df, M, TRIALS, group='B', norepetitions=False, article='Pb'), columns=COLS)
df0 = pd.concat((dfa0, dfb0))
df0['Exposure'] = 'fair'

# half exposure
dfahalf = pd.DataFrame(repeat_propagation(theta_half[-1],df, M, TRIALS, group='A', norepetitions=False, article='Pa'), columns=COLS)
dfbhalf = pd.DataFrame(repeat_propagation(theta_half[1],df, M, TRIALS, group='B', norepetitions=False, article='Pb'), columns=COLS)
dfhalf = pd.concat((dfahalf, dfbhalf))
dfhalf['Exposure'] = 'half'

# optimal (unconstrained)
dfaopt = pd.DataFrame(repeat_propagation(theta_opt[-1],df, M, TRIALS, group='A', norepetitions=False, article='Pa'), columns=COLS)
dfbopt = pd.DataFrame(repeat_propagation(theta_opt[1],df, M, TRIALS, group='B', norepetitions=False, article='Pb'), columns=COLS)
dfopt = pd.concat((dfaopt, dfbopt))
dfopt['Exposure'] = 'opt'

network_results = pd.concat((df0, dfhalf, dfopt))
network_results['N'] = network_results.A + network_results.B
network_results['Setting'] = 'network'
network_results[SAVED_COLS].to_csv('results/abortion_results_network_no_repetitions.csv', index=False)

In [20]:
(pd.concat((network_results[SAVED_COLS],
            model_results[SAVED_COLS]))
     .to_csv('results/abortion_results_no_repetitions.csv', index=False))

### Drop Assumption 3: use actual probabilities instead of sampling from the fitted distributions

In [21]:
# run experiments
COLS = ['Group', 'Article', 'Theta_A', 'Size', 'Seed', 't', 'A', 'B']

# fair (constrained)
dfa0 = pd.DataFrame(repeat_propagation(theta_fair[-1], df, M, TRIALS, group='A', sample_beta=False, article='Pa'), columns=COLS)
dfb0 = pd.DataFrame(repeat_propagation(theta_fair[1], df, M, TRIALS, group='B', sample_beta=False, article='Pb'), columns=COLS)
df0 = pd.concat((dfa0, dfb0))
df0['Exposure'] = 'fair'

# half exposure
dfahalf = pd.DataFrame(repeat_propagation(theta_half[-1], df, M, TRIALS, group='A', sample_beta=False, article='Pa'), columns=COLS)
dfbhalf = pd.DataFrame(repeat_propagation(theta_half[1], df, M, TRIALS, group='B', sample_beta=False, article='Pb'), columns=COLS)
dfhalf = pd.concat((dfahalf, dfbhalf))
dfhalf['Exposure'] = 'half'

# optimal (unconstrained)
dfaopt = pd.DataFrame(repeat_propagation(theta_opt[-1], df, M, TRIALS, group='A', sample_beta=False, article='Pa'), columns=COLS)
dfbopt = pd.DataFrame(repeat_propagation(theta_opt[1], df, M, TRIALS, group='B', sample_beta=False, article='Pb'), columns=COLS)
dfopt = pd.concat((dfaopt, dfbopt))
dfopt['Exposure'] = 'opt'

network_results = pd.concat((df0, dfhalf, dfopt))
network_results['N'] = network_results.A + network_results.B
network_results['Setting'] = 'network'
network_results[SAVED_COLS].to_csv('results/abortion_results_network_probs.csv', index=False)

In [22]:
(pd.concat((network_results[SAVED_COLS],
            model_results[SAVED_COLS]))
     .to_csv('results/abortion_results_probs.csv', index=False))